In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("unsupervised_fraud_results.csv")

df.head()



,step,type,branch,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,unusuallogin,isFlaggedFraud,Acct type,Date of transaction,Time of day,isFraud,Column1,isFraud - Copy,DayOfWeek,DayOfWeek(new)
0,7,PAYMENT,Espana,1526.50,C1908839976,0.0,0.0,M1304522697,0.0,0.0,7,0,Savings,2018-01-03,Afternoon,Safe,7168,0.0,3.0,Wednesday
1,7,PAYMENT,Honduras,1620.17,C70432899,0.0,0.0,M252297858,0.0,0.0,2,0,Savings,2018-01-05,Night,Safe,7211,0.0,5.0,Friday
2,7,PAYMENT,Indonesia,5689.82,C332151172,0.0,0.0,M1430674428,0.0,0.0,3,0,Savings,2018-01-07,Morning,Safe,7268,0.0,7.0,Sunday
3,7,PAYMENT,Estados Unidos,2211.07,C1148172078,0.0,0.0,M650835126,0.0,0.0,13,0,Savings,2018-01-06,Afternoon,Safe,7325,0.0,6.0,Saturday
4,7,PAYMENT,Estados Unidos,824.63,C1142006014,0.0,0.0,M745868137,0.0,0.0,7,0,Savings,2018-01-06,Morning,Safe,7326,0.0,6.0,Saturday


In [5]:
X = df.drop([
    "isFraud",
    "nameOrig",
    "nameDest",
    "Column1",
    "isFraud - Copy",
    "Date of transaction"
], axis=1)

X = pd.get_dummies(X, drop_first=True)
X = X.fillna(0)


In [6]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(
    n_estimators=200,
    contamination=0.01,   # assume 1% fraud
    random_state=42
)

iso.fit(X)


,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",200
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",0.01
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0
,"bootstrap bootstrap: bool, default=FalseIf True, individual trees are fit on random subsets of the trainingdata sampled with replacement. If False, sampling without replacementis performed.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for :meth:`fit`. ``None`` means 1unless in a :obj:`joblib.parallel_backend` context. ``-1`` means usingall processors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo-randomness of the selection of the featureand split values for each branching step and each tree in the forest.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42
,"verbose verbose: int, default=0Controls the verbosity of the tree building process.",0
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fit a wholenew forest. See :term:`the Glossary `... versionadded:: 0.21",False


In [7]:
df["Anomaly"] = iso.predict(X)
df["Anomaly"] = df["Anomaly"].map({1: 0, -1: 1})  # 1 = Fraud


In [8]:
df[df["Anomaly"] == 1].head()


,step,type,branch,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,...,isFlaggedFraud,Acct type,Date of transaction,Time of day,isFraud,Column1,isFraud - Copy,DayOfWeek,DayOfWeek(new),Anomaly
2138,4,CASH_OUT,Panama,10000000.00,C351297720,10000000.0,0.00,C766681183,0.00,9941904.21,...,0,Savings,2018-01-06,Night,Fraud,4441,1.0,6.0,Saturday,1
2198,4,TRANSFER,Cuba,10000000.00,C7162498,12900000.0,2930418.44,C945327594,0.00,0.00,...,0,Current,2018-01-07,Morning,Fraud,4440,1.0,7.0,Sunday,1
5984,7,CASH_IN,Estados Unidos,170689.79,C1442138194,11000000.0,11100000.00,C657736958,5854252.35,5460002.91,...,0,Current,2018-01-07,Morning,Safe,8626,0.0,7.0,Sunday,1
6107,7,CASH_IN,Estados Unidos,25554.54,C1373114222,10700000.0,10700000.00,C935808590,12700000.00,12700000.00,...,0,Savings,2018-01-03,Afternoon,Safe,8624,0.0,3.0,Wednesday,1
6259,1,CASH_IN,Reino Unido,91721.95,C1223303171,7693410.1,7785132.05,C97730845,10000000.00,9940339.29,...,0,Current,2018-01-08,Morning,Safe,2175,0.0,1.0,Monday,1


In [9]:
df["Anomaly"].value_counts()


Anomaly
0    10025
1      102
Name: count, dtype: int64

In [10]:
df.to_csv("unsupervised_fraud_results.csv", index=False)


In [11]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# X already created with get_dummies + fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.values)


In [12]:
from sklearn.ensemble import IsolationForest

iso = IsolationForest(
    n_estimators=300,
    contamination=0.01,   # assume ~1% anomalies
    random_state=42
)

iso.fit(X_scaled)



,"n_estimators n_estimators: int, default=100The number of base estimators in the ensemble.",300
,"max_samples max_samples: ""auto"", int or float, default=""auto""The number of samples to draw from X to train each base estimator.- If int, then draw `max_samples` samples.- If float, then draw `max_samples * X.shape[0]` samples.- If ""auto"", then `max_samples=min(256, n_samples)`.If max_samples is larger than the number of samples provided,all samples will be used for all trees (no sampling).",'auto'
,"contamination contamination: 'auto' or float, default='auto'The amount of contamination of the data set, i.e. the proportionof outliers in the data set. Used when fitting to define the thresholdon the scores of the samples.- If 'auto', the threshold is determined as in the original paper.- If float, the contamination should be in the range (0, 0.5]... versionchanged:: 0.22 The default value of ``contamination`` changed from 0.1 to ``'auto'``.",0.01
,"max_features max_features: int or float, default=1.0The number of features to draw from X to train each base estimator.- If int, then draw `max_features` features.- If float, then draw `max(1, int(max_features * n_features_in_))` features.Note: using a float number less than 1.0 or integer less than number offeatures will enable feature subsampling and leads to a longer runtime.",1.0
,"bootstrap bootstrap: bool, default=FalseIf True, individual trees are fit on random subsets of the trainingdata sampled with replacement. If False, sampling without replacementis performed.",False
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for :meth:`fit`. ``None`` means 1unless in a :obj:`joblib.parallel_backend` context. ``-1`` means usingall processors. See :term:`Glossary ` for more details.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo-randomness of the selection of the featureand split values for each branching step and each tree in the forest.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",42
,"verbose verbose: int, default=0Controls the verbosity of the tree building process.",0
,"warm_start warm_start: bool, default=FalseWhen set to ``True``, reuse the solution of the previous call to fitand add more estimators to the ensemble, otherwise, just fit a wholenew forest. See :term:`the Glossary `... versionadded:: 0.21",False


In [13]:
# 1 = normal, -1 = anomaly
df["Anomaly"] = iso.predict(X_scaled)

# Convert to fraud-style label: 1 = suspected fraud
df["Anomaly"] = df["Anomaly"].map({1: 0, -1: 1})


In [14]:
df["Anomaly"].value_counts()



Anomaly
0    10025
1      102
Name: count, dtype: int64

In [15]:
df[df["Anomaly"] == 1].sort_values("amount", ascending=False).head(10)


,step,type,branch,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,...,isFlaggedFraud,Acct type,Date of transaction,Time of day,isFraud,Column1,isFraud - Copy,DayOfWeek,DayOfWeek(new),Anomaly
2138,4,CASH_OUT,Panama,10000000.00,C351297720,10000000.00,0.00,C766681183,0.00,9941904.21,...,0,Savings,2018-01-06,Night,Fraud,4441,1.0,6.0,Saturday,1
2198,4,TRANSFER,Cuba,10000000.00,C7162498,12900000.00,2930418.44,C945327594,0.00,0.00,...,0,Current,2018-01-07,Morning,Fraud,4440,1.0,7.0,Sunday,1
9188,1,TRANSFER,China,3776389.09,C197491520,0.00,0.00,C1883840933,10100000.00,16900000.00,...,0,Savings,2018-01-08,Afternoon,Safe,1153,0.0,1.0,Monday,1
9137,7,TRANSFER,Haiti,2943845.35,C1360289756,0.00,0.00,C1262822392,18000000.00,22600000.00,...,0,Savings,2018-01-10,Night,Safe,10059,0.0,3.0,Wednesday,1
8937,7,TRANSFER,Italia,2940764.72,C1942806465,0.00,0.00,C1255024717,10200000.00,12900000.00,...,0,Current,2018-01-06,Night,Safe,9635,0.0,6.0,Saturday,1
8173,4,CASH_OUT,Guatemala,2930418.44,C2044690596,2930418.44,0.00,C932583850,3616012.10,6963508.84,...,0,Savings,2018-01-08,Afternoon,Fraud,4443,1.0,1.0,Monday,1
8990,3,TRANSFER,India,2837270.65,C1861416877,0.00,0.00,C97730845,9126860.40,12900000.00,...,0,Savings,2018-01-16,Night,Safe,4155,0.0,2.0,Tuesday,1
8428,1,TRANSFER,Indonesia,2604219.11,C195163481,575667.54,0.00,C97730845,7263554.62,9940339.29,...,0,Savings,2018-01-06,Morning,Safe,1823,0.0,6.0,Saturday,1
8912,7,TRANSFER,Colombia,2576294.80,C1262168655,0.00,0.00,C1880332929,10900000.00,14900000.00,...,0,Current,2018-01-25,Afternoon,Safe,7699,0.0,4.0,Thursday,1
9167,1,TRANSFER,India,2545478.01,C1057507014,0.00,0.00,C1590550415,12400000.00,19200000.00,...,0,Savings,2018-01-16,Morning,Safe,375,0.0,2.0,Tuesday,1


In [16]:
df["Anomaly_Score"] = iso.decision_function(X_scaled)


In [17]:
df.sort_values("Anomaly_Score").head(10)
df.groupby("Anomaly")["amount"].mean()
df.groupby("Anomaly")["unusuallogin"].mean()


Anomaly
0    10.515511
1    10.284314
Name: unusuallogin, dtype: float64

In [18]:
df.to_csv("final_unsupervised_fraud_results.csv", index=False)
print("Saved final_unsupervised_fraud_results.csv")


Saved final_unsupervised_fraud_results.csv
